# BREDA+极小值攻击

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
x_average_resource_quantity_robust1=np.zeros(shape=(100,60000)) #动态一致平均算法追踪平均值的变量
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.1
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")
neighbor_robust1_help=[]
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust1=[] #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [3]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,100-int(proportion_robust1*100)):
    x_average_resource_quantity_robust1[i][0]=agent_resource_quantity_robust1[i][0]
    
#拜占庭用户追踪均值变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    x_average_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

#诚实用户平均成本初始化
'''sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+(1/(100-int(proportion_robust1*100)))*(-beta_robust1[i]*alpha_robust1*math.log(1+agent_resource_quantity_robust1[i][0]))
average_cost_honest_robust1[0][0]=sum_robust1'''

#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust1_help=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            q_robust1_help=q_robust1_help+1
    q_robust1.append(q_robust1_help) 

In [4]:
def BREDA_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,x_average_resource_quantity_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************BREDA+极小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)
        
        #诚实用户对偶变量的迭代
        for i in range(0,100-int(proportion_robust1*100)):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]+iteration_update_robust1*((x_average_resource_quantity_robust1[i][k-1])-(50))
        
        #拜占庭用户对偶变量的迭代
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]-iteration_update_robust1*(((2*a_robust1[i]*agent_resource_quantity_robust1[i][k-1]-2*a_robust1[i]*b_robust1[i])/100)+(dual_variable_robust1[i][k]/100))
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100
                
        #拜占庭用户资源申请量的迭代
        for i in range (100-int((proportion_robust1*100)),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]
                
        #TM鲁棒聚合 每个用户聚合自身以及邻居的动态一致平均值  诚实用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,100-int(proportion_robust1*100)):
            sort_list_robust1=[]
            sum_x_robust1=0
            for j in range (0,15):
                sort_list_robust1.append(x_average_resource_quantity_robust1[neighbor_robust1[i,j]][k-1])
            sort_list_robust1.sort()
            #剔除最大的4个数据和最小的4个数据
            for j in range(0,int(q_robust1[i])):
                del sort_list_robust1[14-j]
            for j in range(0,int(q_robust1[i])):
                del sort_list_robust1[0]     
            for j in range(0,len(sort_list_robust1)):
                sum_x_robust1=sum_x_robust1+sort_list_robust1[j]
            
            x_average_resource_quantity_robust1[i][k]=(1/(16-2*q_robust1[i]))*(x_average_resource_quantity_robust1[i][k-1]+sum_x_robust1)+(agent_resource_quantity_robust1[i][k]-agent_resource_quantity_robust1[i][k-1])      
        
        #拜占庭用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (100-int((proportion_robust1*100)),100):
            x_average_resource_quantity_robust1[i][k]=x_average_resource_quantity_robust1[i][k-1]  
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_robust1[i][k])
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
        
        #iteration_update_robust1=1/(pow(k+1,0.5)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
BREDA_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,x_average_resource_quantity_robust1)

****************************BREDA+极小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 57.914608163588106
agent 2 在 3999 次迭代中的资源申请量为: 57.38430690057768
agent 3 在 3999 次迭代中的资源申请量为: 62.09577572158014
agent 4 在 3999 次迭代中的资源申请量为: 39.51985044897343
agent 5 在 3999 次迭代中的资源申请量为: 59.134516852095395
agent 6 在 3999 次迭代中的资源申请量为: 56.89985857729021
agent 7 在 3999 次迭代中的资源申请量为: 67.11089165373004
agent 8 在 3999 次迭代中的资源申请量为: 48.01438309476713
agent 9 在 3999 次迭代中的资源申请量为: 67.00797937916288
agent 10 在 3999 次迭代中的资源申请量为: 55.61911065972534
agent 11 在 3999 次迭代中的资源申请量为: 67.06493488053916
agent 12 在 3999 次迭代中的资源申请量为: 40.19104556868536
agent 13 在 3999 次迭代中的资源申请量为: 51.40954564016234
agent 14 在 3999 次迭代中的资源申请量为: 42.89457499935852
agent 15 在 3999 次迭代中的资源申请量为: 44.21047704773007
agent 16 在 3999 次迭代中的资源申请量为: 51.114370106799775
agent 17 在 3999 次迭代中的资源申请量为: 51.78811031753629
agent 18 在 3999 次迭代中的资源申请量为: 48.232129543287186
agent 19 在 3999 次迭代中的资源申请量为: 46.57483666300551
agent 20 在 3999 次迭代中的资源申请量为: 52.34221626839073
agent 21 在 3999 次迭代中的资源申请量为: 71.15454235975685
agen

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 57.91407384117629
agent 2 在 5999 次迭代中的资源申请量为: 57.38217127880974
agent 3 在 5999 次迭代中的资源申请量为: 62.09836511819181
agent 4 在 5999 次迭代中的资源申请量为: 39.50047902727417
agent 5 在 5999 次迭代中的资源申请量为: 59.13438864873294
agent 6 在 5999 次迭代中的资源申请量为: 56.897564961991726
agent 7 在 5999 次迭代中的资源申请量为: 67.11922247664853
agent 8 在 5999 次迭代中的资源申请量为: 48.00399644802411
agent 9 在 5999 次迭代中的资源申请量为: 67.01503230138579
agent 10 在 5999 次迭代中的资源申请量为: 55.61652273852224
agent 11 在 5999 次迭代中的资源申请量为: 67.07454630813494
agent 12 在 5999 次迭代中的资源申请量为: 40.172171435370146
agent 13 在 5999 次迭代中的资源申请量为: 51.4034449039423
agent 14 在 5999 次迭代中的资源申请量为: 42.878385522557444
agent 15 在 5999 次迭代中的资源申请量为: 44.196089678197374
agent 16 在 5999 次迭代中的资源申请量为: 51.10760388520873
agent 17 在 5999 次迭代中的资源申请量为: 51.78130080509025
agent 18 在 5999 次迭代中的资源申请量为: 48.222014167732524
agent 19 在 5999 次迭代中的资源申请量为: 46.56379189085494
agent 20 在 5999 次迭代中的资源申请量为: 52.33569195473142
agent 21 在 5999 次迭代中的资源申请量为: 71.16658588813208
agen

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 57.908156142346755
agent 2 在 7999 次迭代中的资源申请量为: 57.37617676662369
agent 3 在 7999 次迭代中的资源申请量为: 62.09252006785915
agent 4 在 7999 次迭代中的资源申请量为: 39.49407064076243
agent 5 在 7999 次迭代中的资源申请量为: 59.12847014102713
agent 6 在 7999 次迭代中的资源申请量为: 56.89157305614596
agent 7 在 7999 次迭代中的资源申请量为: 67.11357731001867
agent 8 在 7999 次迭代中的资源申请量为: 47.997774205964944
agent 9 在 7999 次迭代中的资源申请量为: 67.00933041829515
agent 10 在 7999 次迭代中的资源申请量为: 55.61053127456641
agent 11 在 7999 次迭代中的资源申请量为: 67.06896480156993
agent 12 在 7999 次迭代中的资源申请量为: 40.16578009820174
agent 13 在 7999 次迭代中的资源申请量为: 51.397335271407684
agent 14 在 7999 次迭代中的资源申请量为: 42.8720277378795
agent 15 在 7999 次迭代中的资源申请量为: 44.18978119844495
agent 16 在 7999 次迭代中的资源申请量为: 51.10149809224057
agent 17 在 7999 次迭代中的资源申请量为: 51.77518157429454
agent 18 在 7999 次迭代中的资源申请量为: 48.21579658745299
agent 19 在 7999 次迭代中的资源申请量为: 46.557553972677056
agent 20 在 7999 次迭代中的资源申请量为: 52.32957369248222
agent 21 在 7999 次迭代中的资源申请量为: 71.1610508471996
agent 

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 57.90778292973615
agent 2 在 9999 次迭代中的资源申请量为: 57.375800149943295
agent 3 在 9999 次迭代中的资源申请量为: 62.09214270521493
agent 4 在 9999 次迭代中的资源申请量为: 39.49371363314483
agent 5 在 9999 次迭代中的资源申请量为: 59.12809489791259
agent 6 在 9999 次迭代中的资源申请量为: 56.89119768863148
agent 7 在 9999 次迭代中的资源申请量为: 67.11320329236896
agent 8 在 9999 次迭代中的资源申请量为: 47.99740426513179
agent 9 在 9999 次迭代中的资源申请量为: 67.00895433939168
agent 10 在 9999 次迭代中的资源申请量为: 55.610157165770495
agent 11 在 9999 次迭代中的资源申请量为: 67.068593670734
agent 12 在 9999 次迭代中的资源申请量为: 40.165423232945734
agent 13 在 9999 次迭代中的资源申请量为: 51.39696131321743
agent 14 在 9999 次迭代中的资源申请量为: 42.871664650281545
agent 15 在 9999 次迭代中的资源申请量为: 44.189417046175365
agent 16 在 9999 次迭代中的资源申请量为: 51.10112786143385
agent 17 在 9999 次迭代中的资源申请量为: 51.77480968515186
agent 18 在 9999 次迭代中的资源申请量为: 48.21542612517463
agent 19 在 9999 次迭代中的资源申请量为: 46.55718488132382
agent 20 在 9999 次迭代中的资源申请量为: 52.32920054732657
agent 21 在 9999 次迭代中的资源申请量为: 71.1606782140632
agent 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 57.90777715319197
agent 2 在 51999 次迭代中的资源申请量为: 57.375794208634446
agent 3 在 51999 次迭代中的资源申请量为: 62.092136400172194
agent 4 在 51999 次迭代中的资源申请量为: 39.49371004984924
agent 5 在 51999 次迭代中的资源申请量为: 59.12808893539281
agent 6 在 51999 次迭代中的资源申请量为: 56.8911918548022
agent 7 在 51999 次迭代中的资源申请量为: 67.11319690352695
agent 8 在 51999 次迭代中的资源申请量为: 47.99739931798764
agent 9 在 51999 次迭代中的资源申请量为: 67.00894787227182
agent 10 在 51999 次迭代中的资源申请量为: 55.61015145261939
agent 11 在 51999 次迭代中的资源申请量为: 67.0685874258791
agent 12 在 51999 次迭代中的资源申请量为: 40.16541963313824
agent 13 在 51999 次迭代中的资源申请量为: 51.39695581991356
agent 14 在 51999 次迭代中的资源申请量为: 42.87166049982251
agent 15 在 51999 次迭代中的资源申请量为: 44.189412730231666
agent 16 在 51999 次迭代中的资源申请量为: 51.101122694028945
agent 17 在 51999 次迭代中的资源申请量为: 51.77480439634384
agent 18 在 51999 次迭代中的资源申请量为: 48.215421124769634
agent 19 在 51999 次迭代中的资源申请量为: 46.557180031388285
agent 20 在 51999 次迭代中的资源申请量为: 52.32919514599551
agent 21 在 51999 次迭代中的资源申请量为: 

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 57.90777715319197
agent 2 在 53999 次迭代中的资源申请量为: 57.375794208634446
agent 3 在 53999 次迭代中的资源申请量为: 62.092136400172194
agent 4 在 53999 次迭代中的资源申请量为: 39.49371004984924
agent 5 在 53999 次迭代中的资源申请量为: 59.12808893539281
agent 6 在 53999 次迭代中的资源申请量为: 56.8911918548022
agent 7 在 53999 次迭代中的资源申请量为: 67.11319690352695
agent 8 在 53999 次迭代中的资源申请量为: 47.99739931798764
agent 9 在 53999 次迭代中的资源申请量为: 67.00894787227182
agent 10 在 53999 次迭代中的资源申请量为: 55.61015145261939
agent 11 在 53999 次迭代中的资源申请量为: 67.0685874258791
agent 12 在 53999 次迭代中的资源申请量为: 40.16541963313824
agent 13 在 53999 次迭代中的资源申请量为: 51.39695581991356
agent 14 在 53999 次迭代中的资源申请量为: 42.87166049982251
agent 15 在 53999 次迭代中的资源申请量为: 44.189412730231666
agent 16 在 53999 次迭代中的资源申请量为: 51.101122694028945
agent 17 在 53999 次迭代中的资源申请量为: 51.77480439634384
agent 18 在 53999 次迭代中的资源申请量为: 48.215421124769634
agent 19 在 53999 次迭代中的资源申请量为: 46.557180031388285
agent 20 在 53999 次迭代中的资源申请量为: 52.32919514599551
agent 21 在 53999 次迭代中的资源申请量为: 

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 57.90777715319197
agent 2 在 55999 次迭代中的资源申请量为: 57.375794208634446
agent 3 在 55999 次迭代中的资源申请量为: 62.092136400172194
agent 4 在 55999 次迭代中的资源申请量为: 39.49371004984924
agent 5 在 55999 次迭代中的资源申请量为: 59.12808893539281
agent 6 在 55999 次迭代中的资源申请量为: 56.8911918548022
agent 7 在 55999 次迭代中的资源申请量为: 67.11319690352695
agent 8 在 55999 次迭代中的资源申请量为: 47.99739931798764
agent 9 在 55999 次迭代中的资源申请量为: 67.00894787227182
agent 10 在 55999 次迭代中的资源申请量为: 55.61015145261939
agent 11 在 55999 次迭代中的资源申请量为: 67.0685874258791
agent 12 在 55999 次迭代中的资源申请量为: 40.16541963313824
agent 13 在 55999 次迭代中的资源申请量为: 51.39695581991356
agent 14 在 55999 次迭代中的资源申请量为: 42.87166049982251
agent 15 在 55999 次迭代中的资源申请量为: 44.189412730231666
agent 16 在 55999 次迭代中的资源申请量为: 51.101122694028945
agent 17 在 55999 次迭代中的资源申请量为: 51.77480439634384
agent 18 在 55999 次迭代中的资源申请量为: 48.215421124769634
agent 19 在 55999 次迭代中的资源申请量为: 46.557180031388285
agent 20 在 55999 次迭代中的资源申请量为: 52.32919514599551
agent 21 在 55999 次迭代中的资源申请量为: 

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 57.90777715319197
agent 2 在 57999 次迭代中的资源申请量为: 57.375794208634446
agent 3 在 57999 次迭代中的资源申请量为: 62.092136400172194
agent 4 在 57999 次迭代中的资源申请量为: 39.49371004984924
agent 5 在 57999 次迭代中的资源申请量为: 59.12808893539281
agent 6 在 57999 次迭代中的资源申请量为: 56.8911918548022
agent 7 在 57999 次迭代中的资源申请量为: 67.11319690352695
agent 8 在 57999 次迭代中的资源申请量为: 47.99739931798764
agent 9 在 57999 次迭代中的资源申请量为: 67.00894787227182
agent 10 在 57999 次迭代中的资源申请量为: 55.61015145261939
agent 11 在 57999 次迭代中的资源申请量为: 67.0685874258791
agent 12 在 57999 次迭代中的资源申请量为: 40.16541963313824
agent 13 在 57999 次迭代中的资源申请量为: 51.39695581991356
agent 14 在 57999 次迭代中的资源申请量为: 42.87166049982251
agent 15 在 57999 次迭代中的资源申请量为: 44.189412730231666
agent 16 在 57999 次迭代中的资源申请量为: 51.101122694028945
agent 17 在 57999 次迭代中的资源申请量为: 51.77480439634384
agent 18 在 57999 次迭代中的资源申请量为: 48.215421124769634
agent 19 在 57999 次迭代中的资源申请量为: 46.557180031388285
agent 20 在 57999 次迭代中的资源申请量为: 52.32919514599551
agent 21 在 57999 次迭代中的资源申请量为: 

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 57.90777715319197
agent 2 在 59999 次迭代中的资源申请量为: 57.375794208634446
agent 3 在 59999 次迭代中的资源申请量为: 62.092136400172194
agent 4 在 59999 次迭代中的资源申请量为: 39.49371004984924
agent 5 在 59999 次迭代中的资源申请量为: 59.12808893539281
agent 6 在 59999 次迭代中的资源申请量为: 56.8911918548022
agent 7 在 59999 次迭代中的资源申请量为: 67.11319690352695
agent 8 在 59999 次迭代中的资源申请量为: 47.99739931798764
agent 9 在 59999 次迭代中的资源申请量为: 67.00894787227182
agent 10 在 59999 次迭代中的资源申请量为: 55.61015145261939
agent 11 在 59999 次迭代中的资源申请量为: 67.0685874258791
agent 12 在 59999 次迭代中的资源申请量为: 40.16541963313824
agent 13 在 59999 次迭代中的资源申请量为: 51.39695581991356
agent 14 在 59999 次迭代中的资源申请量为: 42.87166049982251
agent 15 在 59999 次迭代中的资源申请量为: 44.189412730231666
agent 16 在 59999 次迭代中的资源申请量为: 51.101122694028945
agent 17 在 59999 次迭代中的资源申请量为: 51.77480439634384
agent 18 在 59999 次迭代中的资源申请量为: 48.215421124769634
agent 19 在 59999 次迭代中的资源申请量为: 46.557180031388285
agent 20 在 59999 次迭代中的资源申请量为: 52.32919514599551
agent 21 在 59999 次迭代中的资源申请量为: 

In [6]:
np.savetxt('./data/BREDA_TM_oracle_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/BREDA_TM_oracle_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/BREDA_TM_oracle_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/BREDA_TM_oracle_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# BREDA+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
x_average_resource_quantity_robust2=np.zeros(shape=(100,60000)) #动态一致平均算法追踪平均值的变量
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.1
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust2=[] #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,100-int(proportion_robust2*100)):
    x_average_resource_quantity_robust2[i][0]=agent_resource_quantity_robust2[i][0]
    
#拜占庭用户追踪均值变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    x_average_resource_quantity_robust2[i][0]=100    
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust2_help=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            q_robust2_help=q_robust2_help+1
    q_robust2.append(q_robust2_help)

In [10]:
def BREDA_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,x_average_resource_quantity_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************BREDA+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代
        for i in range(0,100-int(proportion_robust2*100)):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]+iteration_update_robust2*((x_average_resource_quantity_robust2[i][k-1])-(50))
        
        #拜占庭用户对偶变量的迭代
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]-iteration_update_robust2*(((2*a_robust2[i]*agent_resource_quantity_robust2[i][k-1]-2*a_robust2[i]*b_robust2[i])/100)+(dual_variable_robust2[i][k]/100))
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100
                
        #拜占庭用户资源申请量的迭代
        for i in range (100-int((proportion_robust2*100)),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]
                
        #TM鲁棒聚合 每个用户聚合自身以及邻居的动态一致平均值  诚实用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,100-int(proportion_robust2*100)):
            sort_list_robust2=[]
            sum_x_robust2=0
            for j in range (0,15):
                sort_list_robust2.append(x_average_resource_quantity_robust2[neighbor_robust2[i,j]][k-1])
            sort_list_robust2.sort()
            #剔除最大的4个数据和最小的4个数据
            for j in range(0,int(q_robust2[i])):
                del sort_list_robust2[14-j]
            for j in range(0,int(q_robust2[i])):
                del sort_list_robust2[0]     
            for j in range(0,len(sort_list_robust2)):
                sum_x_robust2=sum_x_robust2+sort_list_robust2[j]            
            x_average_resource_quantity_robust2[i][k]=(1/(16-2*q_robust2[i]))*(x_average_resource_quantity_robust2[i][k-1]+sum_x_robust2)+(agent_resource_quantity_robust2[i][k]-agent_resource_quantity_robust2[i][k-1])      
        
        #拜占庭用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (100-int((proportion_robust2*100)),100):
            x_average_resource_quantity_robust2[i][k]=x_average_resource_quantity_robust2[i][k-1]           
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_robust2[i][k])
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        #iteration_update_robust2=1/(pow(k+1,0.5)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
BREDA_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,x_average_resource_quantity_robust2)

****************************BREDA+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请量

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 54.07006966139342
agent 2 在 3999 次迭代中的资源申请量为: 53.9849607323027
agent 3 在 3999 次迭代中的资源申请量为: 58.37209329282756
agent 4 在 3999 次迭代中的资源申请量为: 37.011475392702216
agent 5 在 3999 次迭代中的资源申请量为: 55.429633057141494
agent 6 在 3999 次迭代中的资源申请量为: 53.43046753040376
agent 7 在 3999 次迭代中的资源申请量为: 62.85293013141184
agent 8 在 3999 次迭代中的资源申请量为: 45.15664559747478
agent 9 在 3999 次迭代中的资源申请量为: 62.96681607597472
agent 10 在 3999 次迭代中的资源申请量为: 51.995259706341464
agent 11 在 3999 次迭代中的资源申请量为: 62.50114206576936
agent 12 在 3999 次迭代中的资源申请量为: 37.60419542961223
agent 13 在 3999 次迭代中的资源申请量为: 48.30368141533973
agent 14 在 3999 次迭代中的资源申请量为: 40.371347757529946
agent 15 在 3999 次迭代中的资源申请量为: 41.47590953535838
agent 16 在 3999 次迭代中的资源申请量为: 47.7493178543389
agent 17 在 3999 次迭代中的资源申请量为: 48.56283171222158
agent 18 在 3999 次迭代中的资源申请量为: 45.39117347823785
agent 19 在 3999 次迭代中的资源申请量为: 43.76777240670171
agent 20 在 3999 次迭代中的资源申请量为: 49.18563758019835
agent 21 在 3999 次迭代中的资源申请量为: 66.75443801439623
agent 

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 54.05640839448528
agent 2 在 5999 次迭代中的资源申请量为: 53.972225831589505
agent 3 在 5999 次迭代中的资源申请量为: 58.362129065663844
agent 4 在 5999 次迭代中的资源申请量为: 36.98431582669266
agent 5 在 5999 次迭代中的资源申请量为: 55.417183584490985
agent 6 在 5999 次迭代中的资源申请量为: 53.417114091162716
agent 7 在 5999 次迭代中的资源申请量为: 62.84475526442238
agent 8 在 5999 次迭代中的资源申请量为: 45.13772301970409
agent 9 在 5999 次迭代中的资源申请量为: 62.958967504395076
agent 10 在 5999 次迭代中的资源申请量为: 51.980981305002
agent 11 在 5999 次迭代中的资源申请量为: 62.49212597102596
agent 12 在 5999 次迭代中的资源申请量为: 37.577158742414234
agent 13 在 5999 次迭代中的资源申请量为: 48.2881682888226
agent 14 在 5999 次迭代中的资源申请量为: 40.347586633491154
agent 15 在 5999 次迭代中的资源申请量为: 41.45315918783073
agent 16 在 5999 次迭代中的资源申请量为: 47.73177762085706
agent 17 在 5999 次迭代中的资源申请量为: 48.54602013124737
agent 18 在 5999 次迭代中的资源申请量为: 45.37260141533041
agent 19 在 5999 次迭代中的资源申请量为: 43.74830300745679
agent 20 在 5999 次迭代中的资源申请量为: 49.169499830121154
agent 21 在 5999 次迭代中的资源申请量为: 66.74849136232268
age

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 54.049531035316114
agent 2 在 7999 次迭代中的资源申请量为: 53.965409660635196
agent 3 在 7999 次迭代中的资源申请量为: 58.355350439841885
agent 4 在 7999 次迭代中的资源申请量为: 36.97726988486776
agent 5 在 7999 次迭代中的资源申请量为: 55.41034800369156
agent 6 在 7999 次迭代中的资源申请量为: 53.41027615204562
agent 7 在 7999 次迭代中的资源申请量为: 62.837953636693754
agent 8 在 7999 次迭代中的资源申请量为: 45.13080419488911
agent 9 在 7999 次迭代中的资源申请量为: 62.95219853524835
agent 10 在 7999 次迭代中的资源申请量为: 51.97411075852028
agent 11 在 7999 次迭代中的资源申请量为: 62.485270643031676
agent 12 在 7999 次迭代中的资源申请量为: 37.57010884931443
agent 13 在 7999 次迭代中的资源申请量为: 48.28130710085916
agent 14 在 7999 次迭代中的资源申请量为: 40.34059565592306
agent 15 在 7999 次迭代中的资源申请量为: 41.4461715346396
agent 16 在 7999 次迭代中的资源申请量为: 47.724848820915646
agent 17 在 7999 次迭代中的资源申请量为: 48.5391193063003
agent 18 在 7999 次迭代中的资源申请量为: 45.36569094896402
agent 19 在 7999 次迭代中的资源申请量为: 43.74137576485205
agent 20 在 7999 次迭代中的资源申请量为: 49.1626200987971
agent 21 在 7999 次迭代中的资源申请量为: 66.74171175660032
agent

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 54.04910804098922
agent 2 在 9999 次迭代中的资源申请量为: 53.96499073490464
agent 3 在 9999 次迭代中的资源申请量为: 58.35492427869298
agent 4 在 9999 次迭代中的资源申请量为: 36.97688259754283
agent 5 在 9999 次迭代中的资源申请量为: 55.40992516800799
agent 6 在 9999 次迭代中的资源申请量为: 53.40985721864349
agent 7 在 9999 次迭代中的资源申请量为: 62.83751839149226
agent 8 在 9999 次迭代中的资源申请量为: 45.13039965058556
agent 9 在 9999 次迭代中的资源申请量为: 62.9517661178521
agent 10 在 9999 次迭代中的资源申请量为: 51.97369137449667
agent 11 在 9999 次迭代中的资源申请量为: 62.484831904623164
agent 12 在 9999 次迭代中的资源申请量为: 37.569720535488706
agent 13 在 9999 次迭代中的资源申请量为: 48.280895062827014
agent 14 在 9999 次迭代中的资源申请量为: 40.340202087334724
agent 15 在 9999 次迭代中的资源申请量为: 41.44577424230425
agent 16 在 9999 次迭代中的资源申请量为: 47.724436848971365
agent 17 在 9999 次迭代中的资源申请量为: 48.53870785722903
agent 18 在 9999 次迭代中的资源申请量为: 45.36528604211516
agent 19 在 9999 次迭代中的资源申请量为: 43.74097254455059
agent 20 在 9999 次迭代中的资源申请量为: 49.16220840922288
agent 21 在 9999 次迭代中的资源申请量为: 66.74127286026555
agen

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 54.04910045999161
agent 2 在 51999 次迭代中的资源申请量为: 53.96498341471931
agent 3 在 51999 次迭代中的资源申请量为: 58.35491621411036
agent 4 在 51999 次迭代中的资源申请量为: 36.9768783545363
agent 5 在 51999 次迭代中的资源申请量为: 55.40991752284836
agent 6 在 51999 次迭代中的资源申请量为: 53.40984993571277
agent 7 在 51999 次迭代中的资源申请量为: 62.83750952539886
agent 8 在 51999 次迭代中的资源申请量为: 45.1303937818236
agent 9 在 51999 次迭代中的资源申请量为: 62.9517574533503
agent 10 在 51999 次迭代中的资源申请量为: 51.97368411741499
agent 11 在 51999 次迭代中的资源申请量为: 62.4848228187072
agent 12 在 51999 次迭代中的资源申请量为: 37.56971620887758
agent 13 在 51999 次迭代中的资源申请量为: 48.28088844058409
agent 14 在 51999 次迭代中的资源申请量为: 40.340197260477964
agent 15 在 51999 次迭代中的资源申请量为: 41.44576909636568
agent 16 在 51999 次迭代中的资源申请量为: 47.724430348680414
agent 17 在 51999 次迭代中的资源申请量为: 48.5387013585849
agent 18 在 51999 次迭代中的资源申请量为: 45.3652801283219
agent 19 在 51999 次迭代中的资源申请量为: 43.7409668011919
agent 20 在 51999 次迭代中的资源申请量为: 49.16220185433811
agent 21 在 51999 次迭代中的资源申请量为: 66.741263

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 54.04910045999161
agent 2 在 53999 次迭代中的资源申请量为: 53.96498341471931
agent 3 在 53999 次迭代中的资源申请量为: 58.35491621411036
agent 4 在 53999 次迭代中的资源申请量为: 36.9768783545363
agent 5 在 53999 次迭代中的资源申请量为: 55.40991752284836
agent 6 在 53999 次迭代中的资源申请量为: 53.40984993571277
agent 7 在 53999 次迭代中的资源申请量为: 62.83750952539886
agent 8 在 53999 次迭代中的资源申请量为: 45.1303937818236
agent 9 在 53999 次迭代中的资源申请量为: 62.9517574533503
agent 10 在 53999 次迭代中的资源申请量为: 51.97368411741499
agent 11 在 53999 次迭代中的资源申请量为: 62.4848228187072
agent 12 在 53999 次迭代中的资源申请量为: 37.56971620887758
agent 13 在 53999 次迭代中的资源申请量为: 48.28088844058409
agent 14 在 53999 次迭代中的资源申请量为: 40.340197260477964
agent 15 在 53999 次迭代中的资源申请量为: 41.44576909636568
agent 16 在 53999 次迭代中的资源申请量为: 47.724430348680414
agent 17 在 53999 次迭代中的资源申请量为: 48.5387013585849
agent 18 在 53999 次迭代中的资源申请量为: 45.3652801283219
agent 19 在 53999 次迭代中的资源申请量为: 43.7409668011919
agent 20 在 53999 次迭代中的资源申请量为: 49.16220185433811
agent 21 在 53999 次迭代中的资源申请量为: 66.741263

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 54.04910045999161
agent 2 在 55999 次迭代中的资源申请量为: 53.96498341471931
agent 3 在 55999 次迭代中的资源申请量为: 58.35491621411036
agent 4 在 55999 次迭代中的资源申请量为: 36.9768783545363
agent 5 在 55999 次迭代中的资源申请量为: 55.40991752284836
agent 6 在 55999 次迭代中的资源申请量为: 53.40984993571277
agent 7 在 55999 次迭代中的资源申请量为: 62.83750952539886
agent 8 在 55999 次迭代中的资源申请量为: 45.1303937818236
agent 9 在 55999 次迭代中的资源申请量为: 62.9517574533503
agent 10 在 55999 次迭代中的资源申请量为: 51.97368411741499
agent 11 在 55999 次迭代中的资源申请量为: 62.4848228187072
agent 12 在 55999 次迭代中的资源申请量为: 37.56971620887758
agent 13 在 55999 次迭代中的资源申请量为: 48.28088844058409
agent 14 在 55999 次迭代中的资源申请量为: 40.340197260477964
agent 15 在 55999 次迭代中的资源申请量为: 41.44576909636568
agent 16 在 55999 次迭代中的资源申请量为: 47.724430348680414
agent 17 在 55999 次迭代中的资源申请量为: 48.5387013585849
agent 18 在 55999 次迭代中的资源申请量为: 45.3652801283219
agent 19 在 55999 次迭代中的资源申请量为: 43.7409668011919
agent 20 在 55999 次迭代中的资源申请量为: 49.16220185433811
agent 21 在 55999 次迭代中的资源申请量为: 66.741263

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 54.04910045999161
agent 2 在 57999 次迭代中的资源申请量为: 53.96498341471931
agent 3 在 57999 次迭代中的资源申请量为: 58.35491621411036
agent 4 在 57999 次迭代中的资源申请量为: 36.9768783545363
agent 5 在 57999 次迭代中的资源申请量为: 55.40991752284836
agent 6 在 57999 次迭代中的资源申请量为: 53.40984993571277
agent 7 在 57999 次迭代中的资源申请量为: 62.83750952539886
agent 8 在 57999 次迭代中的资源申请量为: 45.1303937818236
agent 9 在 57999 次迭代中的资源申请量为: 62.9517574533503
agent 10 在 57999 次迭代中的资源申请量为: 51.97368411741499
agent 11 在 57999 次迭代中的资源申请量为: 62.4848228187072
agent 12 在 57999 次迭代中的资源申请量为: 37.56971620887758
agent 13 在 57999 次迭代中的资源申请量为: 48.28088844058409
agent 14 在 57999 次迭代中的资源申请量为: 40.340197260477964
agent 15 在 57999 次迭代中的资源申请量为: 41.44576909636568
agent 16 在 57999 次迭代中的资源申请量为: 47.724430348680414
agent 17 在 57999 次迭代中的资源申请量为: 48.5387013585849
agent 18 在 57999 次迭代中的资源申请量为: 45.3652801283219
agent 19 在 57999 次迭代中的资源申请量为: 43.7409668011919
agent 20 在 57999 次迭代中的资源申请量为: 49.16220185433811
agent 21 在 57999 次迭代中的资源申请量为: 66.741263

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 54.04910045999161
agent 2 在 59999 次迭代中的资源申请量为: 53.96498341471931
agent 3 在 59999 次迭代中的资源申请量为: 58.35491621411036
agent 4 在 59999 次迭代中的资源申请量为: 36.9768783545363
agent 5 在 59999 次迭代中的资源申请量为: 55.40991752284836
agent 6 在 59999 次迭代中的资源申请量为: 53.40984993571277
agent 7 在 59999 次迭代中的资源申请量为: 62.83750952539886
agent 8 在 59999 次迭代中的资源申请量为: 45.1303937818236
agent 9 在 59999 次迭代中的资源申请量为: 62.9517574533503
agent 10 在 59999 次迭代中的资源申请量为: 51.97368411741499
agent 11 在 59999 次迭代中的资源申请量为: 62.4848228187072
agent 12 在 59999 次迭代中的资源申请量为: 37.56971620887758
agent 13 在 59999 次迭代中的资源申请量为: 48.28088844058409
agent 14 在 59999 次迭代中的资源申请量为: 40.340197260477964
agent 15 在 59999 次迭代中的资源申请量为: 41.44576909636568
agent 16 在 59999 次迭代中的资源申请量为: 47.724430348680414
agent 17 在 59999 次迭代中的资源申请量为: 48.5387013585849
agent 18 在 59999 次迭代中的资源申请量为: 45.3652801283219
agent 19 在 59999 次迭代中的资源申请量为: 43.7409668011919
agent 20 在 59999 次迭代中的资源申请量为: 49.16220185433811
agent 21 在 59999 次迭代中的资源申请量为: 66.741263

In [12]:
np.savetxt('./data/BREDA_TM_oracle_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/BREDA_TM_oracle_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/BREDA_TM_oracle_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/BREDA_TM_oracle_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量